In [1]:
pip install sentence-transformers==2.7.0 eland elasticsearch transformers


Note: you may need to restart the kernel to use updated packages.


In [2]:
# import modules
from elasticsearch import Elasticsearch
from getpass import getpass
from urllib.request import urlopen
import json
from time import sleep

In [3]:
from pathlib import Path
from eland.ml.pytorch import PyTorchModel
from eland.ml.pytorch.transformers import TransformerModel
from elasticsearch import Elasticsearch
from elasticsearch.client import MlClient

import getpass

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/var/folders/bq/68xyn8mn5q36dzhz2yjpbzkc0000gn/T/ipykernel_70898/3280568807.py:5: DeprecationWarning: Importing from the 'elasticsearch.client' module is deprecated. Instead use 'elasticsearch' module for importing the client.
  from elasticsearch.client import MlClient


In [4]:
es_cloud_id = getpass.getpass('Enter Elastic Cloud ID:  ')
es_user = getpass.getpass('Enter cluster username:  ') 
es_pass = getpass.getpass('Enter cluster password:  ') 

#es_api_id = getpass.getpass('Enter cluster API key ID:  ') 
#es_api_key = getpass.getpass('Enter cluster API key:  ')

Enter Elastic Cloud ID:   ········
Enter cluster username:   ········
Enter cluster password:   ········


In [5]:
#es = Elasticsearch(cloud_id=es_cloud_id, 
#                   api_key=(es_api_id, es_api_key)
#                   )
es = Elasticsearch(cloud_id=es_cloud_id, 
                   basic_auth=(es_user, es_pass)
                   )
es.info() # should return cluster info
     

ObjectApiResponse({'name': 'instance-0000000000', 'cluster_name': 'c35edf6725124dacbb0a80413dd6b79e', 'cluster_uuid': 'Jx3nFk26QaCaovy6upPESQ', 'version': {'number': '8.15.0', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '1a77947f34deddb41af25e6f0ddb8e830159c179', 'build_date': '2024-08-05T10:05:34.233336849Z', 'build_snapshot': False, 'lucene_version': '9.11.1', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [6]:
# Set the model name from Hugging Face and task type
hf_model_id='rufimelo/bert-large-portuguese-cased-sts'
tm = TransformerModel(model_id=hf_model_id, task_type='text_embedding')

#set the modelID as it is named in Elasticsearch
es_model_id = tm.elasticsearch_model_id()

# Download the model from Hugging Face
tmp_path = "models"
Path(tmp_path).mkdir(parents=True, exist_ok=True)
model_path, config, vocab_path = tm.save(tmp_path)

# Load the model into Elasticsearch
ptm = PyTorchModel(es, es_model_id)
ptm.import_model(model_path=model_path, config_path=None, vocab_path=vocab_path, config=config) 


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Invalid model-index. Not loading eval results into CardData.
Asking to pad to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no padding.
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/transformers/modeling_utils.py:4713: FutureWarning: `_is_quantized_training_enabled` is going to be deprecated in transformers 4.39.0. Please use `model.hf_quantizer.is_trainable` instead
  warnings.warn(
`SentenceTransformer._target_device` has been removed, please use `SentenceTransformer.devi

In [7]:
# List the in elasticsearch
m = MlClient.get_trained_models(es, model_id=es_model_id)
m.body

{'count': 1,
 'trained_model_configs': [{'model_id': 'rufimelo__bert-large-portuguese-cased-sts',
   'model_type': 'pytorch',
   'created_by': 'api_user',
   'version': '12.0.0',
   'create_time': 1726834566742,
   'model_size_bytes': 0,
   'estimated_operations': 0,
   'license_level': 'platinum',
   'description': "Model rufimelo/bert-large-portuguese-cased-sts for task type 'text_embedding'",
   'tags': [],
   'metadata': {'per_allocation_memory_bytes': 786483200,
    'per_deployment_memory_bytes': 1333395456},
   'input': {'field_names': ['text_field']},
   'inference_config': {'text_embedding': {'vocabulary': {'index': '.ml-inference-native-000002'},
     'tokenization': {'bert': {'do_lower_case': False,
       'with_special_tokens': True,
       'max_sequence_length': 512,
       'truncate': 'first',
       'span': -1}}}},
   'location': {'index': {'name': '.ml-inference-native-000002'}}}]}

In [8]:
s = MlClient.start_trained_model_deployment(es, model_id=es_model_id)
s.body


{'assignment': {'task_parameters': {'model_id': 'rufimelo__bert-large-portuguese-cased-sts',
   'deployment_id': 'rufimelo__bert-large-portuguese-cased-sts',
   'model_bytes': 1333520644,
   'threads_per_allocation': 1,
   'number_of_allocations': 1,
   'queue_capacity': 1024,
   'cache_size': '1333520644b',
   'priority': 'normal',
   'per_deployment_memory_bytes': 1333395456,
   'per_allocation_memory_bytes': 786483200},
  'routing_table': {},
  'assignment_state': 'starting',
  'start_time': '2024-09-20T12:29:23.138290721Z',
  'max_assigned_allocations': 0}}

In [9]:
stats = MlClient.get_trained_models_stats(es, model_id=es_model_id)
stats.body['trained_model_stats'][0]['deployment_stats']['nodes'][0]['routing_state']
     

IndexError: list index out of range

In [10]:
result = es.search(
 index='search-p28t',
  query={
    'match_all'
  }
 )

result['hits']['hits']

SerializationError: Unable to serialize to JSON: {'query': {'match_all'}} (type: dict)